'Wczytujemy surowa baze, filtrujemy dla Polski i zapisujemy do katalogu 'poland/'


In [3]:
import pandas as pd

full_file_path = 'zrodla/Database to calculate popularity.csv' 

# Wczytanie pełnego pliku CSV do DataFrame
full_df = pd.read_csv(full_file_path, sep=',', low_memory=False)

# Filtracja danych dla Polski
full_df_poland = full_df[full_df['country'] == 'Poland']

# Zapis przefiltrowanych danych do nowego pliku CSV
output_full_csv_path = 'poland/Database to calculate popularity Poland.csv'
full_df_poland.to_csv(output_full_csv_path, index=False)

print("Zapisano przefiltrowane dane do", output_full_csv_path)

Zapisano przefiltrowane dane do poland/Database to calculate popularity Poland.csv


Wczytujemy wynikowa baze filtrujemy dla Polski i zapisujemy do katalogu 'poland/'


In [4]:
import pandas as pd

full_file_path = 'zrodla/Final database.csv' 

# Wczytanie pełnego pliku CSV do DataFrame
full_df = pd.read_csv(full_file_path, sep=',', low_memory=False)

# Filtracja danych dla Polski
full_df_poland = full_df[full_df['Country'] == 'Poland']

# Zapis przefiltrowanych danych do nowego pliku CSV
output_full_csv_path = 'poland/Final database Poland.csv'
full_df_poland.to_csv(output_full_csv_path, index=False)

print("Zapisano przefiltrowane dane do", output_full_csv_path)

Zapisano przefiltrowane dane do poland/Final database Poland.csv


Tak tylko z ciekawosci sprawdzamy ile unikalnych URI jest w wynikowej bazie dla Polski. To nic nie robi wiecej.


In [5]:
import pandas as pd

file_path = 'poland/Final database Poland.csv'
data = pd.read_csv(file_path, sep=',', low_memory=False)

unique_uris_count = data['Uri'].nunique()

print(f"Liczba unikalnych 'uri': {unique_uris_count}")


Liczba unikalnych 'uri': 5273


W pliku wynikowym w kolumnie Popu_max szukamy ktore utwory w Polsce doszly do miejsca od 1 do 50 i zapisujemy uri tych utworow do tymczasowego pliku 'top50.csv'

In [7]:
import pandas as pd

# Załaduj pełny plik 'Final database Poland.csv' 
file_path = 'poland/Final database Poland.csv'
final_db_poland = pd.read_csv(file_path, sep=',', low_memory=False)

# Sortuj dane według kolumny 'Popu_max' w porządku rosnącym
sorted_data = final_db_poland.sort_values(by='Popu_max', ascending=True)

# Wybierz rekordy, gdzie 'Popu_max' wynosi miedzy 1 a 50
top_tracks = sorted_data[sorted_data['Popu_max'].between(1, 50)]

# Wyodrębnij kolumnę 'Uri' i zapisz do nowego DataFrame
top_tracks_uris = top_tracks[['Uri']].copy()

# Zapisz wynikowy DataFrame do pliku CSV
output_file_path = 'poland/top50.csv'
top_tracks_uris.to_csv(output_file_path, index=False)

print(f"Plik 'top50.csv' został zapisany w lokalizacji: {output_file_path}")


Plik 'top50.csv' został zapisany w lokalizacji: poland/top50.csv


Teraz bierzemy dane z glownej bazy szukajac po 'uri' z pliku top50.csv i tworzymy pojedyncze pliki dla kazdego utworu gdzie mamy posortowane rekordy po dacie. Sprawdzamy czy rekordow jest wiecej niz 10, i tylko takie zostawiamy. Kazdy plik nazywamy jak jego uri i zapisujemy do katalogu 'wyniki_uri/'.

In [ ]:
import pandas as pd

popularity_db_path = 'poland/Database to calculate popularity Poland.csv'
popularity_db_poland = pd.read_csv(popularity_db_path, low_memory=False)

top50_path = 'poland/top50.csv' 
top50_uris = pd.read_csv(top50_path)

# Pętla przez każde 'uri' w pliku
for uri in top50_uris['Uri']:
    # Filtrowanie rekordów w bazie danych popularności dla danego 'uri'
    filtered_data = popularity_db_poland[popularity_db_poland['uri'] == uri].copy()

    # Konwersja kolumny 'date' na datetime
    filtered_data.loc[:, 'date'] = pd.to_datetime(filtered_data['date'], format='%d/%m/%Y')

    # Sortowanie przefiltrowanych danych według 'date'
    sorted_filtered_data = filtered_data.sort_values(by='date', ascending=True)

    # Sprawdzenie, czy liczba rekordów wynosi co najmniej 10
    if len(sorted_filtered_data) >= 10:
        # Wyodrębnienie części 'uri'
        uri_part = uri.split('/')[-1]

        # Zapis przefiltrowanych i posortowanych danych do nowego pliku CSV
        output_file_path = f'wyniki_uri/{uri_part}.csv'
        sorted_filtered_data.to_csv(output_file_path, index=False)

print("All files have been processed and saved.")


Usuwamy z katalogu 'wyniki_uri/' pliki z trendem malejacym

In [13]:
import pandas as pd
import os
import numpy as np

# Ścieżka do katalogu z plikami
directory_path = 'wyniki_uri/'

# Pętla przez wszystkie pliki w katalogu
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        
        # Wczytanie danych z pliku
        data = pd.read_csv(file_path)

        # Sprawdzenie, czy 'position' generalnie maleje
        # W tym przypadku, "malejące" oznacza, że średnia różnica między kolejnymi pozycjami jest dodatnia
        # Ponieważ im wyższa liczba w 'position', tym niższa pozycja na liście przebojów
        # takie rozwiazanie zaproponowal mi chatGPT
        if np.mean(np.diff(data['position'])) > 0:
            os.remove(file_path)  # Usunięcie pliku

print("Pliki z malejącym trendem zostały usunięte.")


Pliki z malejącym trendem zostały usunięte.


Rysujemy wykresy :)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

results_directory = 'wyniki_uri/'
images_directory = 'images/'

# Pętla przechodząca przez wszystkie pliki CSV w katalogu wyników
for filename in os.listdir(results_directory):
    if filename.endswith('.csv'):
        # Ścieżka do konkretnego pliku CSV
        file_path = os.path.join(results_directory, filename)

        # Wczytanie danych z pliku CSV
        data = pd.read_csv(file_path)

        # Konwersja kolumny 'date' na format datetime
        data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

        # Rysowanie wykresu liniowego
        plt.figure(figsize=(10, 6))

        # Ustawienie kolorów tła i osi
        plt.gca().set_facecolor('#1a1a1a')
        plt.gca().tick_params(axis='x', colors='lightgray')
        plt.gca().tick_params(axis='y', colors='lightgray')
        plt.gca().spines['bottom'].set_color('white')
        plt.gca().spines['top'].set_color('#1a1a1a')  # Ustawienie na kolor tła, aby ukryć
        plt.gca().spines['right'].set_color('#1a1a1a') # Ustawienie na kolor tła, aby ukryć
        plt.gca().spines['left'].set_color('white')

        # Wykres liniowy
        plt.plot(data['date'], data['position'], marker='o', linestyle='-', color='#49f05b')

        # Obrócenie etykiet daty o 35 stopni i ustawienie ich koloru
        plt.xticks(rotation=35, color='white')
        plt.yticks(color='white')

        # Odwrócenie osi Y, aby pozycja 1 była na górze
        plt.gca().invert_yaxis()

        # Ograniczenie liczby etykiet daty na osi X do 20, jeśli jest ich więcej
        if len(data['date']) > 20:
            plt.gca().xaxis.set_major_locator(plt.MaxNLocator(20))

        plt.ylabel('Pozycja', color='white')

        # Dodanie tytułu z informacjami o utworze
        title = data['title'].iloc[0] if 'title' in data.columns else "Brak tytułu"
        artist = data['artist'].iloc[0] if 'artist' in data.columns else "Brak artysty"
        track = data['track'].iloc[0] if 'track' in data.columns else "Brak ścieżki"
        plt.title(f'Track: {track}\nTitle: {title}\nArtist: {artist}', color='white')

        # Zapisanie wykresu
        plt.savefig(f'{images_directory}/{filename}.png', facecolor='#1a1a1a')

        # Zamykanie wykresu
        plt.close()

print("Wykresy zostały wygenerowane i zapisane.")


Wykresy zostały wygenerowane i zapisane.


A tu wersja generujaca pdf z linkiem :)


In [13]:
pip install reportlab

  Obtaining dependency information for reportlab from https://files.pythonhosted.org/packages/44/2c/f09a5abefa8f2b575acfc19ed4536a365ef0432ae3bce94a57a2de9b4f52/reportlab-4.0.8-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.9 MB 2.6 MB/s eta 0:00:01
   ------- -------------------------------- 0.4/1.9 MB 4.5 MB/s eta 0:00:01
   ------------------ --------------------- 0.9/1.9 MB 7.2 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/1.9 MB 9.2 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 9.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.utils import ImageReader
from PIL import Image

images_directory = 'images/'
pdf_directory = 'pdf/'

# Tworzenie katalogu dla PDF-ów, jeśli nie istnieje
os.makedirs(pdf_directory, exist_ok=True)

# Pętla przechodząca przez wszystkie pliki PNG w katalogu obrazów
for filename in os.listdir(images_directory):
    if filename.endswith('.png'):
        # Ścieżka do obrazu PNG
        image_path = os.path.join(images_directory, filename)

        # Ustalenie rozmiarów obrazu za pomocą PIL
        with Image.open(image_path) as img:
            width, height = img.size

        # Tworzenie pliku PDF
        pdf_path = os.path.join(pdf_directory, filename.replace('.png', '.pdf'))
        c = canvas.Canvas(pdf_path, pagesize=(width, height))
        c.drawImage(ImageReader(image_path), 0, 0, width=width, height=height)

        # Dodawanie linku
        link = "https://open.spotify.com/track/" + filename.replace('.csv.png', '')
        c.linkURL(link, (0, 0, width, height), relative=1)

        c.save()

print("Pliki PDF zostały wygenerowane i zapisane.")


Pliki PDF zostały wygenerowane i zapisane.
